In [2]:
# import os, sys
# from google.colab import drive
# drive.mount('/content/drive')
# my_path = '/content/notebooks'

In [1]:
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from konlpy.tag import Okt
import re
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

okt = Okt()

In [2]:
test_data = pd.read_csv('test_data.csv' , encoding = 'euc-kr')
train_data = pd.read_csv('train_data.csv' , encoding = 'euc-kr')

In [30]:
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)
import konlpy
print(konlpy.__version__)
print(pd.__version__)
print(np.__version__)

2.12.0
2.12.0
0.6.0
1.5.3
1.23.5


In [4]:
# loaded_model = load_model('/content/drive/MyDrive/data_review/best_model.h5')
# print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

# 버전 확인
# pip install --upgrade tensorflow==2.12.0
# pip install --upgrade keras==2.12.0

loaded_model = load_model('best_model2.h5')

In [5]:
# 불용어를 지정하여 필요없는 토큰들은 제거하도록 합니다.

stopwords = ['도', '는', '다', '의', '가', '이', '은', 
             '한', '에', '하', '고', '을', '를', '인', '듯', 
             '과', '와', '네', '들', '듯', '지', '임', '게']

In [6]:
# 단어 집합(vocabulary)의 크기 : 64152
# 등장 빈도가 1번 이하인 희귀 단어의 수: 29480
# 단어 집합에서 희귀 단어의 비율: 45.953360768175585
# 전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.289171484296422

In [7]:
# total_cnt = 64152
# rare_cnt = 29480
# vocab_size = total_cnt - rare_cnt + 2
# tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 

In [8]:
# 긍정 리뷰보다는 부정 리뷰가 좀 더 길게 작성된 경향이 있는 것 같습니다.

X_train = train_data['tokenized'].values
y_train = train_data['label'].values
X_test= test_data['tokenized'].values
y_test = test_data['label'].values

In [9]:
# 정수 인코딩

# 기계가 텍스트를 숫자로 처리할 수 있도록 훈련 데이터와 테스트 데이터에 정수 인코딩을 수행해야 합니다. 
# 훈련 데이터에 대해서 단어 집합(vocaburary)을 만들어봅시다.

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)


In [10]:
total_cnt = 64152
rare_cnt = 29480
vocab_size = total_cnt - rare_cnt + 2

In [25]:
# 단어 집합의 크기는 34674개입니다. 
# 이를 토크나이저의 인자로 넘겨주고, 텍스트 시퀀스를 정수 시퀀스로 변환합니다. 
# 정수 인코딩 과정에서 이보다 큰 숫자가 부여된 단어들은 OOV로 변환하겠습니다.

tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [ ]:
max_len = 330

In [ ]:
# 훈련용 리뷰의 99.99%가 330 이하의 길이를 가집니다. 훈련용 리뷰를 길이 330으로 패딩하겠습니다.

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)


In [15]:
# # test파일 업로드

# from google.colab import files

# # 파일 선택 대화상자 표시
# uploaded = files.upload()

# # 업로드한 파일 확인
# for filename in uploaded.keys():
#     print(f'Uploaded file: {filename}')


In [16]:
# test 파일 업로드 2
#uploaded
# test_re = pd.read_csv('/content/drive/MyDrive/(주)시더스초밥제주연동점_review.csv' , encoding = 'euc-kr')
test_re = pd.read_csv('(주)시더스초밥제주연동점_review.csv' , encoding = 'euc-kr')

In [20]:
# 결측치 처리
test_re = test_re.dropna().reset_index(drop=True)

In [21]:
def sentiment_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = okt.morphs(new_sentence)
    new_sentence = [word for word in new_sentence if not word in stopwords]
    encoded = tokenizer.texts_to_sequences([new_sentence])
    pad_new = pad_sequences(encoded, maxlen = max_len)

    score = float(loaded_model.predict(pad_new))
    if (score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
    else :
        print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))


In [22]:
# test
for num , i in enumerate(test_re.리뷰.tolist()) :
    print(num)
    print(i)
    print(sentiment_predict(i))
    print()

0
원코스요리


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'